In [30]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os

In [31]:
train_images = []
train_labels = []

path = 'diseases_dataset/train'

for disease in os.listdir(f'{path}'):
    for img in os.listdir(f'{path}/{disease}'):
        image = cv2.imread(f'{path}/{disease}/{img}')

        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        
        image = cv2.resize(image, (128, 128))
        image = image.astype('float32') / 255.0

        train_images.append(image)
        train_labels.append(disease)

In [32]:
test_images = []
test_labels = []

path = 'diseases_dataset/test'

for disease in os.listdir(f'{path}'):
    for img in os.listdir(f'{path}/{disease}'):
        image = cv2.imread(f'{path}/{disease}/{img}')

        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        
        image = cv2.resize(image, (128, 128))
        image = image.astype('float32') / 255.0

        test_images.append(image)
        test_labels.append(disease)

In [33]:
train_images_np = np.array(train_images)
test_images_np = np.array(test_images)

In [34]:
train_images_np.shape

(1344, 128, 128, 3)

In [35]:
test_images_np.shape

(140, 128, 128, 3)

In [36]:
from sklearn.preprocessing import LabelEncoder

In [37]:
label = LabelEncoder()

In [38]:
train_labels_lb = label.fit_transform(train_labels)
test_labels_lb = label.transform(test_labels)

In [39]:
from keras.utils import to_categorical

In [40]:
len(set(train_labels))

7

In [41]:
len(set(test_labels))

7

In [42]:
train_labels_cat = to_categorical(train_labels_lb, num_classes=len(set(train_labels)))
test_labels_cat = to_categorical(test_labels_lb, num_classes=len(set(train_labels)))

In [43]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization

In [44]:
from keras.optimizers import Adam

In [45]:
from keras.callbacks import EarlyStopping

In [84]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), input_shape = (128, 128, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(filters = 32, kernel_size = (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))

model.add(Dense(7, activation = 'softmax'))

In [85]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])

In [86]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights = True)

In [87]:
model.fit(train_images_np, train_labels_cat, validation_data=(test_images_np, test_labels_cat), epochs = 100, callbacks=[early_stop])

Epoch 1/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 120ms/step - accuracy: 0.1999 - loss: 1.9723 - val_accuracy: 0.2143 - val_loss: 1.9315
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 120ms/step - accuracy: 0.3362 - loss: 1.7012 - val_accuracy: 0.1429 - val_loss: 1.9283
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.3943 - loss: 1.5690 - val_accuracy: 0.1429 - val_loss: 1.9747
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 117ms/step - accuracy: 0.4235 - loss: 1.4888 - val_accuracy: 0.1429 - val_loss: 1.9548
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 118ms/step - accuracy: 0.5305 - loss: 1.3358 - val_accuracy: 0.1929 - val_loss: 1.9295
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 118ms/step - accuracy: 0.5736 - loss: 1.2103 - val_accuracy: 0.1500 - val_loss: 1.9228
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - accuracy: 0.6010 - loss: 1.1338 - val_accuracy: 0.1857 - val_loss: 1.9726
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - accuracy: 0.6762 - loss: 1.0231 - val_accu

In [182]:
model.save('diseases_helper.h5')